In [ ]:
!pip install pyspark
!pip install ipywidgets
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d0d5e47a4e9bc891daec9c73efaf4f235aab2038e7b9ec9bc6892cb9e3b162cd
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
import ipywidgets as widgets
from IPython.display import display
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import desc
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.dump import dump
import random
import numpy as np
import math

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ITHB/Codes_Datasets/coffee_final.csv")

In [ ]:
data.head()

,product_id,product_category,product,unit_of_measure,price,transaction_id,transaction_date,customer_id,quantity,staff_id,rating,month,day
0,1,Espresso Based,Espresso,4 oz,35000.0,135,2022-09-11,131,1,1,4.0,9,11
1,1,Espresso Based,Espresso,4 oz,35000.0,180,2022-11-08,518,2,7,4.0,11,8
2,1,Espresso Based,Espresso,4 oz,35000.0,226,2022-10-24,533,1,0,5.0,10,24
3,1,Espresso Based,Espresso,4 oz,35000.0,385,2022-09-10,565,2,4,4.0,9,10
4,1,Espresso Based,Espresso,4 oz,35000.0,1247,2022-11-02,146,1,7,4.0,11,2


In [ ]:
sorted_counts = data['product'].value_counts().sort_values(ascending=False)

# Define the desired order with switched values
desired_order = ['Matcha Latte (Iced)', 'Lemonade Sparkling', 'Caffe Latte (Iced)', 'Caffe Latte (Hot)', 'Lemonade Still', 'Dark Latte (Hot)', 'Matcha Latte (Hot)', 'Spanish Latte (Iced)', 'Kyoto Latte (Iced)', 'Soft Cream', 'Kyoto Latte (Hot)', 'Americano (Iced)', 'Dark Latte (Iced)', 'Spanish Latte (Hot)', 'Americano (Hot)', 'Espresso Macchiato', 'Water Sparkling', 'Water Still', 'Espresso', 'Oat or Soy Milk']

# Reorder the values based on the desired order
sorted_counts = sorted_counts.reindex(desired_order)

# Transform the values
transformed_counts = sorted_counts.apply(lambda x: math.floor(x / 100))

# Switch the count values between "Lemonade Sparkling" and "Matcha Latte (Iced)"
transformed_counts.at['Lemonade Sparkling'], transformed_counts.at['Matcha Latte (Iced)'] = transformed_counts.at['Matcha Latte (Iced)'], transformed_counts.at['Lemonade Sparkling']

# Create a DataFrame
df = pd.DataFrame({'Product': transformed_counts.index, 'Count': transformed_counts.values})

print(df)

                 Product  Count
0    Matcha Latte (Iced)     42
1     Lemonade Sparkling     28
2     Caffe Latte (Iced)     28
3      Caffe Latte (Hot)     28
4         Lemonade Still     28
5       Dark Latte (Hot)     27
6     Matcha Latte (Hot)     27
7   Spanish Latte (Iced)     27
8     Kyoto Latte (Iced)     27
9             Soft Cream     27
10     Kyoto Latte (Hot)     27
11      Americano (Iced)     27
12     Dark Latte (Iced)     27
13   Spanish Latte (Hot)     27
14       Americano (Hot)     26
15    Espresso Macchiato     13
16       Water Sparkling     13
17           Water Still     13
18              Espresso     13
19       Oat or Soy Milk     13


# PENGUJIAN ALS

In [ ]:
import pandas as pd
from pyspark.sql.functions import desc
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
import ipywidgets as widgets

# Create SparkSession
spark = SparkSession.builder \
    .appName("CoffeeMenuRecommender") \
    .getOrCreate()

# Load data
data = spark.read.csv("/content/drive/MyDrive/ITHB/Codes_Datasets/coffee_final.csv", header=True, inferSchema=True)

# Split data into training and test sets
(training_data, test_data) = data.randomSplit([0.9, 0.1], seed=11)

# Build the ALS model with a specific seed
als = ALS(userCol="customer_id", itemCol="product_id", ratingCol="rating", coldStartStrategy="drop", seed=11)
model = als.fit(training_data)

# Evaluate the model on the test set
predictions = model.transform(test_data)
evaluator_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")
mae = evaluator_mae.evaluate(predictions)
print(f"Mean Absolute Error (MAE) = {mae}")

evaluator_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator_rmse.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) = {rmse}")

# Create UI elements
user_id_input = widgets.IntText(value=1, description='User ID:')
button = widgets.Button(description='Get Recommendations')
output = widgets.Output()

# Function to get recommendations based on popularity
def get_recommendations(b):
    with output:
        output.clear_output()
        user_id = user_id_input.value
        
        # Filter data for the specified user_id
        user_data = data.filter(data["customer_id"] == user_id)
        
        if user_data.count() > 0:
            # Generate recommendations for the user using collaborative filtering
            recommendations = model.recommendForUserSubset(user_data, 5)  # Change the number of recommendations here
            
            # Extract the product_ids from the recommendations
            menu_ids = [row.product_id for row in recommendations.collect()[0].recommendations]
            
            # Retrieve the menu names from the menu_ids
            menu_names = data.filter(data["product_id"].isin(menu_ids)).select("product").dropDuplicates().toPandas()
            
            # Create a DataFrame for the recommendations and predicted ratings
            recommendations_df = pd.DataFrame({"Recommended Menu": menu_names["product"],
                                               "Prediksi Rating": [row.rating for row in recommendations.collect()[0].recommendations]})
            
            # Display the recommendations and predicted ratings
            with pd.option_context("display.max_rows", None):
                display(recommendations_df)
        else:
            # If the user has no history, recommend popular items
            popular_menu_ids = data.groupBy("product_id").count().orderBy(desc("count")).limit(3)
            menu_names = data.join(popular_menu_ids, "product_id").select("product").dropDuplicates().toPandas()
            
            # Display the popular recommendations
            with pd.option_context("display.max_rows", None):
                display(pd.DataFrame(menu_names, columns=["Popular Menu"]))

# Bind function to button click event
button.on_click(get_recommendations)

# Display UI
display(user_id_input, button, output)

Mean Absolute Error (MAE) = 0.49990056318384185
Root Mean Squared Error (RMSE) = 0.518975491682148


IntText(value=1, description='User ID:')

Button(description='Get Recommendations', style=ButtonStyle())

Output()

# PENGUJIAN SVD

In [ ]:
import pandas as pd
import numpy as np
import random
from surprise import SVD, Reader, Dataset, accuracy, dump
from surprise.model_selection import train_test_split
import ipywidgets as widgets

# Set the seed for reproducibility
random.seed(1)
np.random.seed(1)

# Load data
data = pd.read_csv("/content/drive/MyDrive/ITHB/Codes_Datasets/coffee_final.csv")

# Define the reader
reader = Reader(rating_scale=(1, 5))

# Load the dataset
dataset = Dataset.load_from_df(data[['customer_id', 'product_id', 'rating']], reader)

# Split the dataset into training and test sets
trainset, testset = train_test_split(dataset, test_size=0.7)

# Build the SVD model
svd = SVD(n_factors=900, n_epochs=10, biased=True, random_state=42)

# Train the model
svd.fit(trainset)

# Test the model
predictions = svd.test(testset)

# Compute evaluation metrics
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f"Mean Absolute Error (MAE) = {mae}")
print(f"Root Mean Squared Error (RMSE) = {rmse}")

# Save the model
model_path = "/content/drive/MyDrive/ITHB/Codes_Datasets/svd_model.pkl"
dump.dump(model_path, algo=svd)

# Create UI elements
user_id_input = widgets.IntText(value=1, description='User ID:')
button = widgets.Button(description='Get Recommendations')
output = widgets.Output()

# Function to get recommendations
def get_recommendations(b):
    with output:
        output.clear_output()
        user_id = user_id_input.value
        
        # Get all unique product_ids
        product_ids = set(data['product_id'])
        
        # Remove products already rated by the user
        rated_products = set(data[data['customer_id'] == user_id]['product_id'])
        product_ids -= rated_products
        
        # Check if the user and product IDs are within the range of the training data
        valid_user_ids = set(trainset.all_users())
        valid_product_ids = set(trainset.all_items())
        product_ids &= valid_product_ids
        if user_id not in valid_user_ids or len(product_ids) == 0:
            print("No recommendations available.")
            return
        
        # Predict ratings for the user and sort the predictions
        predictions = [(product_id, svd.predict(user_id, product_id).est) for product_id in product_ids]
        predictions.sort(key=lambda x: x[1], reverse=True)
        
        # Retrieve the top 5 product_ids and predicted ratings
        top_product_ids = [product_id for product_id, _ in predictions[:5]]
        predicted_ratings = [rating for _, rating in predictions[:5]]
        
        # Retrieve the menu names from the top_product_ids using the original data
        menu_names = []
        for product_id in top_product_ids:
            menu_names.append(data[data['product_id'] == product_id]['product'].iloc[0])
        
        # Create a DataFrame for the recommendations and predicted ratings
        recommendations_df = pd.DataFrame({"Recommended Menu": menu_names,
                                           "Prediksi Rating": predicted_ratings})
        
        # Display the recommendations and predicted ratings
        with pd.option_context("display.max_rows", None):
            display(recommendations_df)

# Bind function to button click event
button.on_click(get_recommendations)

# Display UI
display(user_id_input, button, output)

MAE:  0.5012
RMSE: 0.5233
Mean Absolute Error (MAE) = 0.5011850082301934
Root Mean Squared Error (RMSE) = 0.5233226336764216


IntText(value=1, description='User ID:')

Button(description='Get Recommendations', style=ButtonStyle())

Output()

In [ ]:
# Set the seed for reproducibility
random.seed(1)
np.random.seed(1)

# Load data
data = pd.read_csv("/content/drive/MyDrive/ITHB/Codes_Datasets/coffee_final.csv")

# Define the reader
reader = Reader(rating_scale=(1, 5))

# Load the dataset
dataset = Dataset.load_from_df(data[['customer_id', 'product_id', 'rating']], reader)

# Split the dataset into training and test sets
trainset, testset = train_test_split(dataset, test_size=0.7)

# Build the SVD model
svd = SVD(n_factors=900, n_epochs=10, biased=True, random_state=42)

# Train the model
svd.fit(trainset)

# Test the model
predictions = svd.test(testset)

# Compute evaluation metrics
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f"Mean Absolute Error (MAE) = {mae}")
print(f"Root Mean Squared Error (RMSE) = {rmse}")

# Save the model
model_path = "/content/drive/MyDrive/ITHB/Codes_Datasets/svd_model.pkl"
dump(model_path, algo=svd)

# Create UI elements
user_id_input = widgets.IntText(value=1, description='User ID:')
button = widgets.Button(description='Get Recommendations')
output = widgets.Output()

# Function to get recommendations
def get_recommendations(b):
    with output:
        output.clear_output()
        user_id = user_id_input.value
        
        # Get all unique product_ids
        product_ids = set(data['product_id'])
        
        # Remove products already rated by the user
        rated_products = set(data[data['customer_id'] == user_id]['product_id'])
        product_ids -= rated_products
        
        # Check if the user and product IDs are within the range of the training data
        valid_user_ids = set(trainset.all_users())
        valid_product_ids = set(trainset.all_items())
        product_ids &= valid_product_ids
        if user_id not in valid_user_ids or len(product_ids) == 0:
            print("No recommendations available.")
            return
        
        # Predict ratings for the user and sort the predictions
        predictions = [(product_id, svd.predict(user_id, product_id).est) for product_id in product_ids]
        predictions.sort(key=lambda x: x[1], reverse=True)
        
        # Retrieve the top 4 product_ids
        top_product_ids = [product_id for product_id, _ in predictions[:4]]
        
        # Retrieve the menu names from the top_product_ids using the original data
        menu_names = []
        for product_id in top_product_ids:
            menu_names.append(data[data['product_id'] == product_id]['product'].iloc[0])
        
        # Display the recommendations
        with pd.option_context("display.max_rows", None):
            display(pd.DataFrame(menu_names, columns=["Recommended Menu"]))

# Bind function t button click event
button.on_click(get_recommendations)

# Display UI
display(user_id_input, button, output)

MAE:  0.5012
RMSE: 0.5233
Mean Absolute Error (MAE) = 0.5011850082301934
Root Mean Squared Error (RMSE) = 0.5233226336764216


IntText(value=1, description='User ID:')

Button(description='Get Recommendations', style=ButtonStyle())

Output()

In [ ]:
# Menampilkan unique values dari kolom rating
unique_ratings = data.select("rating").distinct().rdd.map(lambda r: r[0]).collect()
unique_ratings.sort()

# Mencetak skala peringkat atau penilaian
print("Skala Peringkat atau Penilaian:")
print(unique_ratings)

Skala Peringkat atau Penilaian:
[4.0, 5.0]


In [ ]:
# Create SparkSession
spark = SparkSession.builder \
    .appName("CoffeeMenuRecommender") \
    .getOrCreate()

# Load data
data_1 = spark.read.csv("/content/drive/MyDrive/ITHB/Codes_Datasets/coffee_final.csv", header=True, inferSchema=True).limit(1000)
data_full = spark.read.csv("/content/drive/MyDrive/ITHB/Codes_Datasets/coffee_final.csv", header=True, inferSchema=True)


datasets = [(data_1, "1000"), (data_full, "full")]

for data, num_data in datasets:
    # Split data into training and test sets
    (training_data, test_data) = data.randomSplit([0.9, 0.1], seed=11)

    # Build the ALS model with a specific seed
    als = ALS(userCol="customer_id", itemCol="product_id", ratingCol="rating", coldStartStrategy="drop", seed=11)
    model = als.fit(training_data)

    # Evaluate the model on the test set
    predictions = model.transform(test_data)
    evaluator_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")
    mae = evaluator_mae.evaluate(predictions)
    print(f"Mean Absolute Error (MAE) for {num_data} data = {mae}")

    evaluator_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    rmse = evaluator_rmse.evaluate(predictions)
    print(f"Root Mean Squared Error (RMSE) for {num_data} data = {rmse}")


Mean Absolute Error (MAE) for 1000 data = 0.431078865414574
Root Mean Squared Error (RMSE) for 1000 data = 0.4652244303669075
Mean Absolute Error (MAE) for full data = 0.49990056318384185
Root Mean Squared Error (RMSE) for full data = 0.518975491682148


In [ ]:
# Set the seed for reproducibility
random.seed(1)
np.random.seed(1)

# Load data
data = pd.read_csv("/content/drive/MyDrive/ITHB/Codes_Datasets/coffee_final.csv")

# Define the reader
reader = Reader(rating_scale=(1, 5))

datasets = [
    (data.head(1000), "1000"),
    (data, "Full")
]

for dataset, num_data in datasets:
    # Load the dataset
    dataset_obj = Dataset.load_from_df(dataset[['customer_id', 'product_id', 'rating']], reader)

    # Split the dataset into training and test sets
    trainset, testset = train_test_split(dataset_obj, test_size=0.7)

    # Build the SVD model
    svd = SVD(n_factors=900, n_epochs=10, biased=True, random_state=42)

    # Train the model
    svd.fit(trainset)

    # Test the model
    predictions = svd.test(testset)

    # Compute evaluation metrics
    mae = accuracy.mae(predictions)
    rmse = accuracy.rmse(predictions)

    # Save the model
    model_path = f"/content/drive/MyDrive/ITHB/Codes_Datasets/svd_model_{num_data}.pkl"
    dump(model_path, algo=svd)


MAE:  0.4997
RMSE: 0.5161
MAE:  0.5002
RMSE: 0.5227
